# Demo Wine sans MLflow

In [17]:
import warnings
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import numpy as np
warnings.filterwarnings("ignore")
np.random.seed(40)


In [18]:
import pandas as pd 

# read data from file
df = pd.read_csv("wine-quality.csv")

In [19]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(df)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# make it easy to pass datasets
datasets = { 'train_x': train_x, 'train_y': train_y, 'test_x': test_x, 'test_y': test_y }

In [30]:
shapes = [ "%s : %s" % (name, dataset.shape) for (name,dataset) in datasets.items() ]
print(shapes)

['train_x : (3673, 11)', 'train_y : (3673, 1)', 'test_x : (1225, 11)', 'test_y : (1225, 1)']


In [31]:
def eval_parameters(in_alpha, in_l1_ratio):
    # Set default values if no alpha is provided
    alpha = float(in_alpha) if not in_alpha is None else 0.5
    # Set default values if no l1_ratio is provided
    l1_ratio = float(in_l1_ratio) if not in_l1_ratio is None else 0.5
    return alpha, l1_ratio

In [32]:
def eval_metrics(actual, predicted):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2


In [33]:
def train_elasticnet(in_alpha, in_l1_ratio):
    from sklearn.linear_model import ElasticNet

    alpha, l1_ratio = eval_parameters(in_alpha, in_l1_ratio)
    print("Parameters (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    
    # train with ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    return rmse

In [34]:
train_elasticnet(0.5, 0.5) # alpha 0.5, L1 0.5

Parameters (alpha=0.500000, l1_ratio=0.500000):
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689


0.82224284975954

In [35]:
train_elasticnet(0.5, 0.4)

Parameters (alpha=0.500000, l1_ratio=0.400000):
Elasticnet model (alpha=0.500000, l1_ratio=0.400000):
  RMSE: 0.8134651568415164
  MAE: 0.6249674191990106
  R2: 0.1453313314484228


0.8134651568415164

In [36]:
train_elasticnet(0.4, 0.4)

Parameters (alpha=0.400000, l1_ratio=0.400000):
Elasticnet model (alpha=0.400000, l1_ratio=0.400000):
  RMSE: 0.8052999730885096
  MAE: 0.6225920385560693
  R2: 0.16240275256684844


0.8052999730885096